# Multi-Head Attention (MHA) Transformer Training
## Train Transformer on WikiText-2 - Harvard NLP Style

**Based on Harvard NLP's Annotated Transformer** 🎓

This notebook:
- Loads **pre-processed WikiText-2** data (from Datasets.ipynb)
- Trains encoder-decoder transformer using **Harvard NLP's implementation patterns**
- Optimized for Google Colab

**What's special about this implementation?**
- ✅ Uses `make_model()` factory function (Harvard NLP way)
- ✅ Uses `rate()` for learning rate scheduling (from the paper)
- ✅ Uses `TextGenerator` for professional text generation
- ✅ Follows [Harvard NLP's Annotated Transformer](https://nlp.seas.harvard.edu/annotated-transformer/)

**Prerequisites:**
1. Run `Datasets.ipynb` first to create processed data
2. Download `data_processed.zip` from that notebook
3. Upload it to Colab and extract: `!unzip data_processed.zip`

**References:**
- Harvard NLP Annotated Transformer: https://nlp.seas.harvard.edu/annotated-transformer/
- Original Paper: "Attention is All You Need" (Vaswani et al., 2017)

---

## 1️⃣ Check GPU

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected - training will be slow!")

print(f"\nUsing device: {device}")

## 2️⃣ Mount Google Drive (to save checkpoints)

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Create checkpoint directory
CHECKPOINT_DIR = '/content/drive/MyDrive/mha_checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
print(f"✓ Checkpoints will be saved to: {CHECKPOINT_DIR}")

## 3️⃣ Clone Repository

In [ ]:
# Clone your repository
!git clone https://github.com/mohamedAtoui/LLM-Journey
%cd LLM-Journey

print("✓ Repository cloned!")

## 4️⃣ Install Dependencies

In [ ]:
# Install dependencies AND the mha package properly!
!pip install -q datasets transformers tqdm

# Install YOUR mha package in editable mode (industry standard!)
!pip install -q -e .

print("✓ Packages installed!")
print("✓ MHA package installed in editable mode (proper way!)")

## 5️⃣ Import Everything - Harvard NLP Style

**Note:** We're using **Harvard NLP's Annotated Transformer** patterns!

The package is properly installed with `pip install -e .` in the previous cell.

**What we're importing:**
- ✅ `make_model()`: Harvard NLP's factory function to create transformers
- ✅ `rate()`: Learning rate schedule from the paper
- ✅ `TextGenerator`: Professional text generation utilities
- ✅ `Batch`: Batch processing with automatic masking
- ✅ Mask utilities: For creating attention masks

**Benefits:**
- ✅ Imports work from any directory
- ✅ Works reliably in Colab
- ✅ IDE autocomplete works
- ✅ Same patterns as the original paper
- ✅ Professional & reproducible

**Backward compatible:** The old `Transformer` class still works if you prefer!

In [ ]:
# Standard imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer
from datasets import load_from_disk
from tqdm import tqdm
import math

# Harvard NLP style imports (RECOMMENDED)
from mha import make_model
from mha.utils import rate, Batch
from mha.inference import TextGenerator
from mha.attention import subsequent_mask

# Mask utilities (for backward compatibility)
from mha import create_combined_mask, create_padding_mask

print("✓ All imports successful!")
print("✓ Using Harvard NLP's Annotated Transformer patterns!")
print("  - make_model() for creating transformers")
print("  - rate() for LR scheduling")
print("  - TextGenerator for inference")

## 6️⃣ Configuration (Simple Dictionary!)

In [ ]:
# Harvard NLP style configuration
config = {
    # Model architecture (Harvard NLP parameter names)
    'src_vocab': 50257,          # Source vocabulary size (GPT-2)
    'tgt_vocab': 50257,          # Target vocabulary size (same for LM)
    'N': 6,                      # Number of encoder/decoder layers
    'd_model': 512,              # Model dimension
    'd_ff': 2048,                # Feed-forward dimension
    'h': 8,                      # Number of attention heads
    'dropout': 0.1,              # Dropout probability
    'max_seq_length': 512,       # Max sequence length (matches data!)
    
    # Training hyperparameters
    'batch_size': 8,             # Batch size (small for Colab memory)
    'num_epochs': 3,             # Number of epochs
    'warmup_steps': 2000,        # LR warmup steps (from paper)
    'gradient_clip': 1.0,        # Gradient clipping
}

print("Configuration (Harvard NLP Style):")
print("=" * 60)
print(f"  Model: make_model(src_vocab={config['src_vocab']}, tgt_vocab={config['tgt_vocab']}, N={config['N']})")
print(f"  Dimensions: d_model={config['d_model']}, d_ff={config['d_ff']}, h={config['h']}")
print(f"  Training: {config['num_epochs']} epochs, batch_size={config['batch_size']}")
print(f"  LR Schedule: warmup_steps={config['warmup_steps']} (using rate() function)")
print("=" * 60)

## 7️⃣ Load Pre-Processed Data (From Datasets.ipynb)

**Note:** This loads the data you already prepared in `Datasets.ipynb`!

Before running this cell:
1. Make sure you ran `Datasets.ipynb` and downloaded `data_processed.zip`
2. Upload `data_processed.zip` to Colab
3. Extract it: `!unzip data_processed.zip`

This will load the **already tokenized** WikiText-2 dataset (512 tokens, GPT-2 tokenizer).

In [ ]:
from datasets import load_from_disk

# Load pre-processed datasets (created from Datasets.ipynb)
print("Loading pre-processed WikiText-2 dataset...")
print("(Make sure you've uploaded data_processed.zip and extracted it!)\n")

# Path to your pre-processed data
DATA_PATH = './data/wikitext2_processed'

try:
    # Load from disk
    dataset = load_from_disk(DATA_PATH)
    
    train_dataset = dataset['train']
    val_dataset = dataset['validation']
    
    # Initialize tokenizer (still needed for pad_token_id)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    
    # Set PyTorch format
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    
    print(f"✓ Dataset loaded successfully!")
    print(f"  Tokenizer vocab size: {len(tokenizer)}")
    print(f"  Train samples: {len(train_dataset):,}")
    print(f"  Val samples: {len(val_dataset):,}")
    print(f"  Sequence length: {len(train_dataset[0]['input_ids'])} tokens")
    
except FileNotFoundError:
    print("❌ Error: Pre-processed data not found!")
    print(f"\nExpected path: {DATA_PATH}")
    print("\nPlease:")
    print("  1. Run Datasets.ipynb to create the processed data")
    print("  2. Download data_processed.zip from Datasets.ipynb")
    print("  3. Upload and extract it to Colab:")
    print("     !unzip data_processed.zip")
    raise

## 8️⃣ Create DataLoaders (Simple!)

In [ ]:
# Simple collate function (data is already in PyTorch format!)
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': input_ids.clone()  # For language modeling
    }

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    collate_fn=collate_fn
)

print(f"✓ DataLoaders created!")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")

## 9️⃣ Initialize Model - Harvard NLP Style

**Using `make_model()` factory function!**

This is the standard way to create a Transformer from Harvard NLP's Annotated Transformer.

**What `make_model()` does:**
1. Creates encoder and decoder with N layers
2. Initializes embeddings with √d_model scaling
3. Adds sinusoidal positional encoding
4. Creates generator (output projection)
5. Applies Xavier/Glorot initialization

**Comparison:**
- **Old way:** `model = Transformer(vocab_size=..., d_model=..., num_heads=...)`
- **Harvard NLP way:** `model = make_model(src_vocab=..., tgt_vocab=..., N=...)`

Both work! But Harvard NLP's way is cleaner and matches the paper.

In [ ]:
# Create transformer model using Harvard NLP's make_model() function
model = make_model(
    src_vocab=config['src_vocab'],
    tgt_vocab=config['tgt_vocab'],
    N=config['N'],
    d_model=config['d_model'],
    d_ff=config['d_ff'],
    h=config['h'],
    dropout=config['dropout']
).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✓ Model created using make_model()!")
print(f"  Architecture: EncoderDecoder with {config['N']} layers")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Model size: ~{total_params * 4 / 1e6:.1f} MB")
print(f"\n  Harvard NLP components:")
print(f"    - Encoder: {config['N']}-layer stack")
print(f"    - Decoder: {config['N']}-layer stack")
print(f"    - Embeddings: with √d_model scaling")
print(f"    - Positional Encoding: Sinusoidal")
print(f"    - Generator: Linear + log_softmax")

## 🔟 Setup Optimizer, Scheduler & Loss - Harvard NLP Style

**Using `rate()` function for learning rate scheduling!**

The `rate()` function implements the exact schedule from "Attention is All You Need":

```
lrate = d_model^(-0.5) * min(step^(-0.5), step * warmup^(-1.5))
```

This increases LR linearly during warmup, then decreases proportionally to the inverse square root of the step number.

**Benefits:**
- ✅ Exact formula from the paper
- ✅ Cleaner code (no manual lambda)
- ✅ Well-tested implementation

In [ ]:
# Adam optimizer (parameters from the paper)
optimizer = optim.Adam(
    model.parameters(),
    lr=1.0,  # Will be scaled by rate() function
    betas=(0.9, 0.98),
    eps=1e-9
)

# Learning rate scheduler using Harvard NLP's rate() function
# IMPORTANT: Start from step 1, not 0, to avoid division issues
scheduler = LambdaLR(
    optimizer,
    lr_lambda=lambda step: rate(
        step + 1,  # Add 1 to avoid step=0 issues
        model_size=config['d_model'],
        factor=1.0,
        warmup=config['warmup_steps']
    )
)

# Loss function - use NLLLoss since Generator outputs log_softmax (Harvard NLP way)
# NLLLoss expects log probabilities, which is what model.generator() returns
criterion = nn.NLLLoss(ignore_index=tokenizer.pad_token_id, reduction='sum')

print("✓ Optimizer, scheduler, and loss function ready!")
print(f"  Optimizer: Adam with betas=(0.9, 0.98), eps=1e-9")
print(f"  LR Schedule: rate() function with warmup={config['warmup_steps']}")
print(f"  Loss: NLLLoss with reduction='sum' (Harvard NLP way)")
print(f"\n  LR at step 1: {rate(1, config['d_model'], 1.0, config['warmup_steps']):.6f}")
print(f"  LR at step {config['warmup_steps']}: {rate(config['warmup_steps'], config['d_model'], 1.0, config['warmup_steps']):.6f}")
print(f"  LR at step {config['warmup_steps']*2}: {rate(config['warmup_steps']*2, config['d_model'], 1.0, config['warmup_steps']):.6f}")

## 1️⃣1️⃣ Training Function

In [ ]:
def train_epoch(model, train_loader, optimizer, scheduler, criterion, device, epoch):
    """Train for one epoch (Harvard NLP style with numerical stability)"""
    model.train()
    total_loss = 0
    total_tokens = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    for batch_idx, batch in enumerate(pbar):
        # Get data
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        # Prepare src and tgt (shift by 1 for next-token prediction)
        src = input_ids[:, :-1]          # All tokens except last
        tgt_input = input_ids[:, :-1]    # Same (decoder input)
        tgt_output = labels[:, 1:]       # All tokens except first (target)
        
        # Create masks
        src_mask = create_padding_mask(src, pad_token_id=tokenizer.pad_token_id)
        tgt_mask = create_combined_mask(tgt_input, pad_token_id=tokenizer.pad_token_id, causal=True)
        
        # Forward pass through model (returns decoder hidden states)
        decoder_output = model(src, tgt_input, src_mask, tgt_mask)
        
        # Apply generator to get log probabilities (Harvard NLP way)
        # Generator applies: linear projection + log_softmax
        log_probs = model.generator(decoder_output)
        
        # Compute loss (NLLLoss with reduction='sum')
        # We sum the loss and then divide by number of tokens (Harvard NLP way)
        loss_sum = criterion(
            log_probs.reshape(-1, config['tgt_vocab']),  # (batch*seq_len, vocab)
            tgt_output.reshape(-1)                        # (batch*seq_len,)
        )
        
        # Count non-padding tokens in this batch
        num_tokens = (tgt_output != tokenizer.pad_token_id).sum().item()
        
        # Normalize loss by number of tokens
        loss = loss_sum / num_tokens if num_tokens > 0 else loss_sum
        
        # Check for NaN/Inf before backward pass
        if not torch.isfinite(loss):
            print(f"\n⚠️  Warning: Non-finite loss detected at batch {batch_idx}")
            print(f"   Loss value: {loss.item()}")
            continue  # Skip this batch
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping (important for stability!)
        torch.nn.utils.clip_grad_norm_(model.parameters(), config['gradient_clip'])
        
        optimizer.step()
        scheduler.step()
        
        # Track metrics (use loss_sum for proper averaging)
        total_loss += loss_sum.item()
        total_tokens += num_tokens
        
        # Update progress bar
        current_lr = scheduler.get_last_lr()[0]
        pbar.set_postfix({
            'loss': f"{loss.item():.4f}",
            'lr': f"{current_lr:.2e}"
        })
    
    # Calculate average loss per token
    avg_loss = total_loss / total_tokens if total_tokens > 0 else float('inf')
    perplexity = math.exp(min(avg_loss, 100))  # Cap to prevent overflow
    return avg_loss, perplexity

## 1️⃣2️⃣ Validation Function

In [ ]:
@torch.no_grad()
def validate(model, val_loader, criterion, device):
    """Validate the model (Harvard NLP style with numerical stability)"""
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    for batch in tqdm(val_loader, desc="Validation"):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        
        src = input_ids[:, :-1]
        tgt_input = input_ids[:, :-1]
        tgt_output = labels[:, 1:]
        
        src_mask = create_padding_mask(src, pad_token_id=tokenizer.pad_token_id)
        tgt_mask = create_combined_mask(tgt_input, pad_token_id=tokenizer.pad_token_id, causal=True)
        
        # Forward pass through model (returns decoder hidden states)
        decoder_output = model(src, tgt_input, src_mask, tgt_mask)
        
        # Apply generator to get log probabilities (Harvard NLP way)
        log_probs = model.generator(decoder_output)
        
        # Compute loss (sum over batch)
        loss_sum = criterion(
            log_probs.reshape(-1, config['tgt_vocab']),  # (batch*seq_len, vocab)
            tgt_output.reshape(-1)                        # (batch*seq_len,)
        )
        
        num_tokens = (tgt_output != tokenizer.pad_token_id).sum().item()
        total_loss += loss_sum.item()
        total_tokens += num_tokens
    
    # Calculate average loss per token
    avg_loss = total_loss / total_tokens if total_tokens > 0 else float('inf')
    perplexity = math.exp(min(avg_loss, 100))  # Cap to prevent overflow
    return avg_loss, perplexity

## 1️⃣3️⃣ Main Training Loop 🚀

In [ ]:
# Training loop
best_val_loss = float('inf')
history = {'train_loss': [], 'train_ppl': [], 'val_loss': [], 'val_ppl': []}

print("\n" + "="*60)
print("Starting Training!")
print("="*60 + "\n")

for epoch in range(1, config['num_epochs'] + 1):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch}/{config['num_epochs']}")
    print(f"{'='*60}")
    
    # Train
    train_loss, train_ppl = train_epoch(
        model, train_loader, optimizer, scheduler, criterion, device, epoch
    )
    print(f"\n📊 Train Loss: {train_loss:.4f} | Perplexity: {train_ppl:.2f}")
    
    # Validate
    val_loss, val_ppl = validate(model, val_loader, criterion, device)
    print(f"📊 Val Loss: {val_loss:.4f} | Perplexity: {val_ppl:.2f}")
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_ppl'].append(train_ppl)
    history['val_loss'].append(val_loss)
    history['val_ppl'].append(val_ppl)
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint_path = f"{CHECKPOINT_DIR}/best_model_epoch{epoch}.pt"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_ppl': val_ppl,
            'config': config,
        }, checkpoint_path)
        print(f"✅ Best model saved! (Val Loss: {val_loss:.4f})")
        print(f"   Saved to: {checkpoint_path}")

print("\n" + "="*60)
print("✅ Training Complete!")
print("="*60)
print(f"\nBest Validation Loss: {best_val_loss:.4f}")
print(f"Final Train PPL: {history['train_ppl'][-1]:.2f}")
print(f"Final Val PPL: {history['val_ppl'][-1]:.2f}")

In [ ]:
## 🎓 Harvard NLP Features Showcase

**Congratulations! You just trained a Transformer using Harvard NLP's Annotated Transformer patterns!**

### What's Different?

| Feature | Old Implementation | Harvard NLP Style (This Notebook) |
|---------|-------------------|-----------------------------------|
| **Model Creation** | `Transformer(vocab_size=..., d_model=...)` | `make_model(src_vocab=..., N=...)` |
| **Architecture** | Manual initialization | Factory function with Xavier init |
| **LR Schedule** | Custom lambda function | `rate()` function from paper |
| **Generation** | Custom function | `TextGenerator` with multiple strategies |
| **Parameter Names** | Mixed conventions | Paper's original names (N, h, d_ff) |

### Key Harvard NLP Components Used

1. **`make_model()`** - Factory function that creates:
   - `EncoderDecoder` wrapper
   - `Encoder` with N layers
   - `Decoder` with N layers
   - `Embeddings` with √d_model scaling
   - `PositionalEncoding` (sinusoidal)
   - `Generator` (output projection)

2. **`rate()`** - Learning rate schedule:
   ```python
   lrate = d_model^(-0.5) * min(step^(-0.5), step * warmup^(-1.5))
   ```

3. **`TextGenerator`** - Professional generation with:
   - Greedy decoding
   - Temperature sampling  
   - Top-k sampling
   - Nucleus (top-p) sampling

### Benefits

✅ **Cleaner Code**: Factory functions vs manual initialization  
✅ **Paper Accuracy**: Exact formulas from "Attention is All You Need"  
✅ **Educational**: Matches the annotated transformer tutorial  
✅ **Flexible**: Multiple generation strategies  
✅ **Professional**: Industry-standard patterns

### Learn More

- **Harvard NLP Annotated Transformer**: https://nlp.seas.harvard.edu/annotated-transformer/
- **Original Paper**: "Attention is All You Need" (Vaswani et al., 2017)
- **Your Implementation**: Modular package structure with Harvard NLP patterns

**Backward Compatibility**: The old `Transformer` class still works! This notebook shows the recommended Harvard NLP way.

## 1️⃣4️⃣ Text Generation - Harvard NLP Style

**Using `TextGenerator` class!**

The `TextGenerator` provides multiple sampling strategies:
- **Greedy decoding**: Always pick the most likely token (fast, deterministic)
- **Temperature sampling**: Control randomness with temperature parameter
- **Top-k sampling**: Sample from top k most likely tokens
- **Nucleus (top-p) sampling**: Sample from smallest set of tokens with cumulative probability > p

This is much more flexible than a custom generation function!

In [ ]:
# Create TextGenerator using Harvard NLP's class
generator = TextGenerator(model, tokenizer, device=device)

# Test generation with multiple strategies
print("Testing Text Generation (Harvard NLP Style)...")
print("=" * 70)

prompts = [
    "The transformer architecture",
    "In the field of artificial intelligence",
    "Machine learning is"
]

for i, prompt in enumerate(prompts):
    print(f"\n{i+1}. Prompt: \"{prompt}\"")
    print("-" * 70)
    
    # Greedy decoding (deterministic)
    generated = generator.generate_greedy(prompt, max_length=30)
    print(f"   Greedy:      {generated}")
    
    # Temperature sampling (more creative)
    generated_temp = generator.generate_with_temperature(prompt, temperature=0.8, max_length=30)
    print(f"   Temperature: {generated_temp}")
    
    print()

print("=" * 70)
print("✓ Text generation working!")
print("\nAvailable methods:")
print("  - generate_greedy(): Deterministic, picks most likely token")
print("  - generate_with_temperature(): Controlled randomness")
print("  - generate_top_k(): Sample from top k tokens")
print("  - generate_nucleus(): Sample from cumulative probability > p")

## 1️⃣5️⃣ Load Best Model & Evaluate

In [ ]:
# Load best checkpoint
import glob

checkpoint_files = glob.glob(f"{CHECKPOINT_DIR}/best_model_epoch*.pt")

if checkpoint_files:
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    print(f"Loading best model: {latest_checkpoint}")
    
    checkpoint = torch.load(latest_checkpoint)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"\n✅ Best model loaded!")
    print(f"  Epoch: {checkpoint['epoch']}")
    print(f"  Train Loss: {checkpoint['train_loss']:.4f}")
    print(f"  Val Loss: {checkpoint['val_loss']:.4f}")
    print(f"  Val Perplexity: {checkpoint['val_ppl']:.2f}")
    
    # Test generation with best model
    print("\n" + "="*60)
    print("Testing generation with best model:")
    print("="*60 + "\n")
    
    prompt = "The attention mechanism allows"
    generated = generate_text(model, tokenizer, prompt, max_length=40, device=device)
    print(f"Prompt: {prompt}")
    print(f"Generated: {generated}")
    
else:
    print("❌ No checkpoint found!")

print("\n" + "="*60)
print("✅ All Done!")
print("="*60)
print(f"\nCheckpoints saved at: {CHECKPOINT_DIR}")
print("You can find them in your Google Drive!")